In [ ]:
pip install "inferedge_moss>=1.0.0b15" dspy

In [2]:
import asyncio
import dspy
from dspy.dsp.utils import dotdict
from dspy.primitives.prediction import Prediction
from dspy.retrievers.retrieve import Retrieve
from inferedge_moss import DocumentInfo, MossClient, QueryOptions
import nest_asyncio
import os
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

def run_async(coro):
    """Helper to run async coroutines in a notebook environment."""
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    if loop.is_running():
        return loop.run_until_complete(coro)
    else:
        return asyncio.run(coro)

class MossRM(Retrieve):
    """A retrieval module that uses Moss (InferEdge) to return the top passages for a given query.

    Args:
        index_name (str): The name of the Moss index.
        moss_client (MossClient): An instance of the Moss client.
        k (int, optional): The default number of top passages to retrieve. Default to 3.

    Examples:
        Below is a code snippet that shows how to use Moss as the default retriever:
        ```python
        from inferedge_moss import MossClient
        import dspy

        moss_client = MossClient("your-project-id", "your-project-key")
        retriever_model = MossRM("my_index_name", moss_client=moss_client)
        dspy.configure(rm=retriever_model)

        retrieve = dspy.Retrieve(k=1)
        topK_passages = retrieve("what are the stages in planning, sanctioning and execution of public works").passages
        ```
    """

    def __init__(
        self,
        index_name: str,
        moss_client: MossClient,
        k: int = 3,
        alpha: float = 0.5,
    ):
        self._index_name = index_name
        self._moss_client = moss_client
        self._alpha = alpha

        super().__init__(k=k)

    def forward(self, query_or_queries: str | list[str], k: int | None = None, **kwargs) -> Prediction:
        """Search with Moss for self.k top passages for query or queries.

        Args:
            query_or_queries (Union[str, list[str]]): The query or queries to search for.
            k (Optional[int]): The number of top passages to retrieve. Defaults to self.k.
            kwargs : Additional arguments for Moss client.

        Returns:
            dspy.Prediction: An object containing the retrieved passages.
        """
        k = k if k is not None else self.k
        queries = [query_or_queries] if isinstance(query_or_queries, str) else query_or_queries
        queries = [q for q in queries if q]
        passages = []

        for query in queries:
            options = QueryOptions(top_k=k, alpha=self._alpha, **kwargs)
            # Since MossClient methods are async, we use asyncio.run to call them synchronously.
            # This assumes the loop is not already running, which is typical for DSPy RM calls.
            result = asyncio.run(self._moss_client.query(self._index_name, query, options=options))

            for doc in result.docs:
                passages.append(
                    dotdict({"long_text": doc.text, "id": doc.id, "metadata": doc.metadata, "score": doc.score})
                )

        return passages

    def get_objects(self, num_samples: int = 5) -> list[dict]:
        """Get objects from Moss."""
        # Note: Moss's get_docs might return all docs or have limits.
        # Here we attempt to fetch and return up to num_samples.
        result = asyncio.run(self._moss_client.get_docs(self._index_name))
        # result is likely a list of DocumentInfo or similar
        objects = []
        for i, doc in enumerate(result):
            if i >= num_samples:
                break
            objects.append({"id": doc.id, "text": doc.text, "metadata": doc.metadata})
        return objects

    def insert(self, new_object_properties: dict | list[dict]):
        """Insert one or more objects into Moss."""
        if isinstance(new_object_properties, dict):
            new_object_properties = [new_object_properties]

        docs = [DocumentInfo(**props) for props in new_object_properties]
        asyncio.run(self._moss_client.add_docs(self._index_name, docs))

In [3]:
import requests
import os
import asyncio
from inferedge_moss import MossClient, DocumentInfo
# 2. Initialize MossClient (Replace with your actual keys)
MOSS_PROJECT_ID = os.getenv("MOSS_PROJECT_ID", os.getenv("MOSS_PROJECT_ID"))
MOSS_PROJECT_KEY = os.getenv("MOSS_PROJECT_KEY", os.getenv("MOSS_PROJECT_KEY"))

client = MossClient(MOSS_PROJECT_ID, MOSS_PROJECT_KEY)
INDEX_NAME = "moss-cookbook"

llm = dspy.LM(model="gpt-4.1-mini",api_key="sk-proj")
# retriever_model = MossRM(INDEX_NAME, moss_client=client) ways to use Moss as retriever
# dspy.settings.configure(lm=llm, rm=retriever_model)
dspy.configure(lm=llm)

In [ ]:
# Create a ReAct agent using Moss as a tool
def moss_search(query: str):
    """Searches the Moss knowledge base for relevant passages."""
    
    # 2. Configure query options
    options = QueryOptions(top_k=TOP_K, alpha=0)
    # 3. Use the run_async helper to call Moss from a sync function
    # (Results are automatically returned as a list of documents)
    results = run_async(client.query(INDEX, query, options=options))
    # You can process multiple topK results as well
    if results.docs:
        print(f"[Tool DEBUG] Found {len(results.docs)} results for: {query}")
        print(f"[Tool DEBUG] First snippet: {results.docs[0].text}...")
    
    if not results.docs:
        return "No relevant info found."
        
    return "\n".join([f"- {doc.text}" for doc in results.docs])


INDEX = "moss-cookbook"
TOP_K = 10
client.load_index(INDEX)

# Initialize the ReAct agent with the Moss search tool
react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[moss_search],
    max_iters=5
)

# Example usage
question = "whats the shipping address?"
result = react_agent(question=question)

print(f"Answer: {result.answer}")
print("\nTool calls made (Trajectory):")
for step in result.trajectory:
    print(step)

/var/folders/ny/7jk1wzqd7wz6z964crstxgqm0000gn/T/ipykernel_89530/729364508.py:22: RuntimeWarning: coroutine 'MossClient.load_index' was never awaited
  client.load_index(INDEX)


[Tool DEBUG] Found 10 results for: shipping address
[Tool DEBUG] First snippet: How can I change my shipping address? Contact our customer service team....
Answer: The shipping address is near St. Mark's Square.

Tool calls made (Trajectory):
thought_0
tool_name_0
tool_args_0
observation_0
thought_1
tool_name_1
tool_args_1
observation_1
